In [1]:
!pip install shap

In [2]:
import os
import random
import requests
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model, model_selection
import shap

import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader

import torchvision
from torchvision import transforms
from torchvision.utils import make_grid
from torchvision.models import resnet18
import torch.nn.functional as F
import torch.nn.utils.prune as prune
from copy import deepcopy
from math import sqrt
from torch.utils.data import Subset

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Running on device:", DEVICE.upper())

# manual random seed is used for dataset partitioning
# to ensure reproducible results across runs
RNG = torch.Generator().manual_seed(42)

# Download dataset

In [3]:
# download and pre-process CIFAR10
normalize = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ]
)

train_set = torchvision.datasets.CIFAR10(
    root="./data", train=True, download=True, transform=normalize
)
train_loader = DataLoader(train_set, batch_size=128, shuffle=True, num_workers=2)

class_indices = [[] for _ in range(10)]

for idx, (_, label) in enumerate(train_set):
    class_indices[label].append(idx)

print(class_indices)

# we split held out data into test and validation set
held_out = torchvision.datasets.CIFAR10(
    root="./data", train=False, download=True, transform=normalize
)
test_set, val_set = torch.utils.data.random_split(held_out, [0.5, 0.5], generator=RNG)
print(test_set)
test_loader = DataLoader(test_set, batch_size=128, shuffle=False, num_workers=2)
val_loader = DataLoader(val_set, batch_size=128, shuffle=False, num_workers=2)

test_class_indices = [[] for _ in range(10)]

for idx, (_, label) in enumerate(test_set):
    test_class_indices[label].append(idx)

print(test_class_indices)

# Znalezienie indeksów dla klasy 0 i pozostałych klas
indices_class_0 = test_class_indices[0]  # Indeksy próbek klasy 0
indices_other_classes = [idx for i in range(1, 10) for idx in test_class_indices[i]]  # Indeksy pozostałych klas

# Tworzenie podzbiorów danych
test_set_0 = Subset(test_set, indices_class_0)
test_set_no_0 = Subset(test_set, indices_other_classes)

# Tworzenie DataLoaderów
test_loader_0 = DataLoader(test_set_0, batch_size=128, shuffle=False, num_workers=2)
test_loader_no_0 = DataLoader(test_set_no_0, batch_size=128, shuffle=False, num_workers=2)

100%|██████████| 170M/170M [00:13<00:00, 12.9MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data


KeyboardInterrupt: 

In [ ]:
print(test_set)

# Starting model

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Pobieramy predefiniowany model ResNet18
model = resnet18(weights=None)  # Można użyć `weights='IMAGENET1K_V1'`, ale lepiej trenować od zera
model.fc = nn.Linear(512, 10)  # Dostosowanie warstwy końcowej do 10 klas CIFAR-10
model = model.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
num_epochs = 40  # Liczba epok

for epoch in range(num_epochs):
    running_loss = 0.0
    model.train()  # Przełącz na tryb trenowania

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()  # Zerowanie gradientów
        outputs = model(images)  # Forward pass
        loss = criterion(outputs, labels)  # Obliczenie straty
        loss.backward()  # Backpropagation
        optimizer.step()  # Aktualizacja wag

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

print("Trening zakończony!")

In [ ]:
def full_retraining(train_loader):
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

  # Pobieramy predefiniowany model ResNet18
  model = resnet18(weights=None)  # Można użyć `weights='IMAGENET1K_V1'`, ale lepiej trenować od zera
  model.fc = nn.Linear(512, 10)  # Dostosowanie warstwy końcowej do 10 klas CIFAR-10
  model = model.to(device)

  criterion = nn.CrossEntropyLoss()
  optimizer = optim.Adam(model.parameters(), lr=0.001)

  num_epochs = 40  # Liczba epok

  for epoch in range(num_epochs):
      running_loss = 0.0
      model.train()  # Przełącz na tryb trenowania

      for images, labels in train_loader:
          images, labels = images.to(device), labels.to(device)

          optimizer.zero_grad()  # Zerowanie gradientów
          outputs = model(images)  # Forward pass
          loss = criterion(outputs, labels)  # Obliczenie straty
          loss.backward()  # Backpropagation
          optimizer.step()  # Aktualizacja wag

          running_loss += loss.item()

      print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

  print("Trening zakończony!")
  model.eval()
  return model

In [ ]:
model.eval()  # Przełączamy model na tryb ewaluacji
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy na zbiorze testowym: {accuracy:.2f}%')

In [ ]:
def data_to_forget(train, class_indices, label, size):
  sampled_class = []
  if label == -1:
    for i in range(10):
      print("*")
      tmp = random.sample(class_indices[i], int(size/10))
      class_indices[i] = list(set(class_indices[i]) - set(tmp))
      sampled_class.extend(tmp)
  else:
    sampled_class = random.sample(class_indices[label], size)  # Wylosowanie 50 próbek
    class_indices[label] = list(set(class_indices[label]) - set(sampled_class))

  flat_class_indices = [idx for sublist in class_indices for idx in sublist]
  forget_set = torch.utils.data.Subset(train_set, sampled_class)
  retain_set = torch.utils.data.Subset(train_set, flat_class_indices)

  forget_loader = torch.utils.data.DataLoader(
      forget_set, batch_size=128, shuffle=True, num_workers=2
  )
  retain_loader = torch.utils.data.DataLoader(
      retain_set, batch_size=128, shuffle=True, num_workers=2, generator=RNG
  )
  return forget_loader, retain_loader, class_indices

#  Fauchan - 1st place



In [ ]:
def evaluation(net, dataloader, criterion, device = 'cuda'): ##evaluation function
    net.eval()
    total_samp = 0
    total_acc = 0
    total_loss = 0.0
    for sample in dataloader:
        images, labels = sample['image'].to(device), sample['age_group'].to(device)
        _pred = net(images)
        total_samp+=len(labels)
        #print(f'total_samp={total_samp}')
        loss = criterion(_pred, labels)
        total_loss += loss.item()
        total_acc+=(_pred.max(1)[1] == labels).float().sum().item()
        #print(f'total_acc={total_acc}')
    #print(f'total_sample={total_samp}')
    mean_loss = total_loss / len(dataloader)
    mean_acc = total_acc/total_samp
    print(f'loss={mean_loss}')
    print(f'acc={mean_acc}')
    return

In [ ]:
USE_MOCK: bool = False

from torch.optim.lr_scheduler import CosineAnnealingLR,CosineAnnealingWarmRestarts,StepLR
def kl_loss_sym(x,y):
    kl_loss = nn.KLDivLoss(reduction='batchmean')
    return kl_loss(nn.LogSoftmax(dim=-1)(x),y)
def fauchan(
        net,
        retain_loader,
        forget_loader,
        val_loader,
):
    """Simple unlearning by finetuning."""
    print('-----------------------------------')
    epochs = 8
    retain_bs = 256
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=0.005,
                          momentum=0.9, weight_decay=0)
    optimizer_retain = optim.SGD(net.parameters(), lr=0.001*retain_bs/64, momentum=0.9, weight_decay=1e-2)
    ##the learning rate is associated with the batchsize we used
    optimizer_forget = optim.SGD(net.parameters(), lr=3e-4, momentum=0.9, weight_decay=0)
    total_step = int(len(forget_loader)*epochs)
    retain_ld = DataLoader(retain_loader.dataset, batch_size=retain_bs, shuffle=True)
    retain_ld4fgt = DataLoader(retain_loader.dataset, batch_size=256, shuffle=True)
    scheduler = CosineAnnealingLR(optimizer_forget, T_max=total_step, eta_min=1e-6)
    """if USE_MOCK: ##Use some Local Metric as reference
        net.eval()
        print('Forget')
        evaluation(net, forget_loader, criterion)
        print('Valid')
        evaluation(net, validation_loader, criterion)"""
    net.train()
    for inputs, targets in forget_loader: ##First Stage
        inputs = inputs.to(DEVICE)
        optimizer.zero_grad()
        outputs = net(inputs)
        uniform_label = torch.ones_like(outputs).to(DEVICE) / outputs.shape[1] ##uniform pseudo label
        loss = kl_loss_sym(outputs, uniform_label) ##optimize the distance between logits and pseudo labels
        loss.backward()
        optimizer.step()
    """if USE_MOCK:
        print('Forget')
        evaluation(net,forget_loader,criterion)
        print('Valid')
        evaluation(net, validation_loader,criterion)
        print(f'epoch={epochs} and retain batch_sz={retain_bs}')"""
    net.train()
    for ep in range(epochs): ##Second Stage
        net.train()
        for (inputs_forget, outputs_forget), (inputs_retain, outputs_retain) in zip(forget_loader, retain_ld4fgt):##Forget Round
            t = 1.15 ##temperature coefficient
            inputs_forget, inputs_retain = inputs_forget.to(DEVICE), inputs_retain.to(DEVICE)
            optimizer_forget.zero_grad()
            outputs_forget,outputs_retain = net(inputs_forget),net(inputs_retain).detach()
            loss = (-1 * nn.LogSoftmax(dim=-1)(outputs_forget @ outputs_retain.T/t)).mean() ##Contrastive Learning loss
            loss.backward()
            optimizer_forget.step()
            scheduler.step()
        for inputs, labels in retain_ld: ##Retain Round
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            optimizer_retain.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer_retain.step()
        """if USE_MOCK:
            print(f'epoch {ep}:')
            print('Retain')
            evaluation(net, retain_ld, criterion)
            print('Forget')
            evaluation(net, forget_loader, criterion)
            print('Valid')
            evaluation(net, validation_loader, criterion)"""
    print('-----------------------------------')
    return net

# Kookmin - 2 place

In [ ]:
sch = 'linear'
init_rate = 0.3
init_method = 'snip_little_grad'
lr = 0.001
epoch = 5
weight_decay = 5e-4

In [ ]:
from torch.optim.lr_scheduler import _LRScheduler

class LinearAnnealingLR(_LRScheduler):
    def __init__(self, optimizer, num_annealing_steps, num_total_steps):
        self.num_annealing_steps = num_annealing_steps
        self.num_total_steps = num_total_steps

        super().__init__(optimizer)

    def get_lr(self):
        if self._step_count <= self.num_annealing_steps:
            return [base_lr * self._step_count / self.num_annealing_steps for base_lr in self.base_lrs]
        else:
            return [base_lr * (self.num_total_steps - self._step_count) / (self.num_total_steps - self.num_annealing_steps) for base_lr in self.base_lrs]

In [ ]:
class Masker(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x, mask):
        ctx.save_for_backward(mask)
        return x

    @staticmethod
    def backward(ctx, grad):
        mask, = ctx.saved_tensors
        return grad * mask, None


class MaskConv2d(nn.Conv2d):
    def __init__(self, mask, in_channels, out_channels, kernel_size, stride=1,
                 padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros', device='cpu'):
        super(MaskConv2d, self).__init__(in_channels, out_channels, kernel_size, stride,
                                     padding, dilation, groups, bias, padding_mode, device=device)
        self.mask = mask

    def forward(self, input):
        masked_weight = Masker.apply(self.weight, self.mask)
        return super(MaskConv2d, self)._conv_forward(input, masked_weight, self.bias)

@torch.no_grad()
def re_init_model_random_zero_grad_Maskconv(model, px):
    print("Apply Unstructured random re init no grads Globally (all conv layers)")
    for name, m in list(model.named_modules()):
        if isinstance(m, nn.Conv2d):
            mask = torch.zeros_like(m.weight, device=DEVICE).bool()
            nparams_toprune = round(px*mask.nelement())

            prob = torch.rand_like(mask.float(), device=DEVICE)
            topk = torch.topk(prob.view(-1), k=nparams_toprune)
            mask.view(-1)[topk.indices] = True

            new_conv = MaskConv2d(mask, m.in_channels, m.out_channels, m.kernel_size, m.stride,
                 m.padding, m.dilation, m.groups, m.bias!=None, m.padding_mode, device=DEVICE)
            #nn.init.kaiming_normal_(new_conv.weight, mode="fan_out", nonlinearity="relu")
            new_conv.weight.data[~mask] = m.weight[~mask]
            setattr(model, name, new_conv)

@torch.no_grad()
def re_init_model_random_little_grad_Maskconv_fanout(model, px):
    print("Apply Unstructured re_init_model_random_little_grad_Maskconv_fanout (all conv layers)")
    for name, m in list(model.named_modules()):
        if isinstance(m, nn.Conv2d):
            mask = torch.zeros_like(m.weight, device=DEVICE).bool()
            nparams_toprune = round(px*mask.nelement())

            prob = torch.rand_like(mask.float(), device=DEVICE)
            topk = torch.topk(prob.view(-1), k=nparams_toprune)
            mask.view(-1)[topk.indices] = True
            grad_mask = mask.clone().float()
            grad_mask[grad_mask==0] += 0.1

            new_conv = MaskConv2d(grad_mask, m.in_channels, m.out_channels, m.kernel_size, m.stride,
                 m.padding, m.dilation, m.groups, m.bias!=None, m.padding_mode, device=DEVICE)
            nn.init.kaiming_normal_(new_conv.weight, mode="fan_out", nonlinearity="relu")
            new_conv.weight.data[~mask] = m.weight[~mask]
            setattr(model, name, new_conv)

In [ ]:
@torch.no_grad()
def re_init_model_snip_ver2_little_grad(model, px): # re init smallest gradients
    print("Apply Unstructured re_init_model_snip_ver2_little_grad Globally (all conv layers)")
    for name, m in list(model.named_modules()):
        if isinstance(m, nn.Conv2d):
            mask = torch.zeros_like(m.weight, device=DEVICE).bool()
            nparams_toprune = round(px*mask.nelement())

            out_c, in_c, ke, _ = mask.shape
            value = -m.weight.grad.abs()
            topk = torch.topk(value.view(-1), k=nparams_toprune)
            mask.view(-1)[topk.indices] = True
            grad_mask = mask.clone().float()
            grad_mask[grad_mask==0] += 0.1

            new_conv = MaskConv2d(grad_mask, m.in_channels, m.out_channels, m.kernel_size, m.stride,
                 m.padding, m.dilation, m.groups, m.bias!=None, m.padding_mode, device=DEVICE)
            nn.init.kaiming_normal_(new_conv.weight, mode="fan_out", nonlinearity="relu")

            new_conv.weight.data[~mask] = m.weight[~mask]

            set_layer(model, name, new_conv)

def set_layer(model, layer_name, layer):
    splited = layer_name.split('.')
    if len(splited) == 1:
        setattr(model, splited[0], layer)
    elif len(splited) == 3:
        setattr(getattr(model, splited[0])[int(splited[1])], splited[2], layer)
    elif len(splited) == 4:
        getattr(getattr(model, splited[0])[int(splited[1])], splited[2])[int(splited[3])] = layer

@torch.no_grad()
def replace_maskconv(model):
    print("Remove Maskconv")
    for name, m in list(model.named_modules()):
        if isinstance(m, MaskConv2d):
            conv = nn.Conv2d(m.in_channels, m.out_channels, m.kernel_size, m.stride,
                 m.padding, m.dilation, m.groups, m.bias!=None, m.padding_mode, device=DEVICE)
            conv.weight.data = m.weight
            conv.bias = m.bias
            set_layer(model, name, conv)

In [ ]:
@torch.no_grad()
def re_init_model_snip_ver2(model, px): # re init smallest gradients
    print("Apply Unstructured snip ve2 re init no grads Globally (all conv layers)")
    for name, m in model.named_modules():
        if isinstance(m, nn.Conv2d):
            mask = torch.zeros_like(m.weight, device=DEVICE).bool()
            nparams_toprune = round(px*mask.nelement())

            out_c, in_c, ke, _ = mask.shape
            value = -m.weight.grad.abs()
            topk = torch.topk(value.view(-1), k=nparams_toprune)
            mask.view(-1)[topk.indices] = True

            m.weight.data[mask] = nn.Conv2d(in_c, out_c, ke, device=DEVICE).weight[mask]

def get_grads_for_snip(model, retain_loader, forget_loader):
    indices = torch.randperm(len(retain_loader.dataset), dtype=torch.int32, device='cpu')[:len(forget_loader.dataset)]

    model.zero_grad()
    for inputs, targets in retain_loader:
        inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)

        outputs = model(inputs)
        loss = F.cross_entropy(outputs, targets)
        loss.backward()

    for inputs, targets in  forget_loader:
        inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)

        outputs = model(inputs)
        loss = -F.cross_entropy(outputs, targets)
        loss.backward()

In [ ]:
def kookmin(
    net,
    retain_loader,
    forget_loader,
    val_loader):
    if init_method=='snip_little_grad':
        replace_maskconv(net)
        get_grads_for_snip(net, retain_loader, forget_loader)
        re_init_model_snip_ver2_little_grad(net, init_rate)
    else:
        raise "not implemented"

    """Simple unlearning by finetuning."""
    epochs = epoch
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=lr,
                      momentum=0.9, weight_decay=weight_decay)
    if sch=='cosine':
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
    elif sch=='linear':
        scheduler = LinearAnnealingLR(optimizer, num_annealing_steps=(epochs+1)//2, num_total_steps=epochs+1)
    elif sch=='decrease':
        print('decrease')
        scheduler = LinearAnnealingLR(optimizer, num_annealing_steps=1, num_total_steps=epochs+1)

    net.train()

    for ep in range(epochs):
        for inputs, targets in retain_loader:
            inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)

            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            loss.backward()
            optimizer.step()

        scheduler.step()
    #remove_prune(net)
    net.eval()
    print("Kookmin finished")
    return net

# Seif - 3rd place

In [ ]:
def seif(
    net,
    retain_loader,
    forget_loader,
    val_loader):
    """Simple unlearning by finetuning."""



    class CustomCrossEntropyLoss(nn.Module):
        def __init__(self, class_weights=None):
            super(CustomCrossEntropyLoss, self).__init__()
            self.class_weights = class_weights

        def forward(self, input, target):
            # Compute the standard cross-entropy loss
            ce_loss = nn.functional.cross_entropy(input, target)

            # Apply class weights to the loss if provided
            if self.class_weights is not None:
                # Calculate the weights for each element in the batch based on the target
                weights = torch.tensor([self.class_weights[i] for i in target], device=input.device)
                ce_loss = torch.mean(ce_loss * weights)

            return ce_loss



    # Define the vision_confuser function
    def vision_confuser(model, std = 0.6):
        for name, module in model.named_children():
            if hasattr(module, 'weight'):
                if 'conv' in name:

                    actual_value = module.weight.clone().detach()
                    new_values = torch.normal(mean=actual_value, std=std)
                    module.weight.data.copy_(new_values)

    vision_confuser(net)

    epochs = 4

    w = 0.05

    class_weights = [1, w, w, w, w, w, w, w, w, w]
    criterion = CustomCrossEntropyLoss(class_weights)


    optimizer = optim.SGD(net.parameters(), lr=0.0007,
                      momentum=0.9, weight_decay=5e-4)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
        optimizer, T_max=epochs)

    net.train()
    i=0

    for ep in range(epochs):
        i=0
        net.train()
        for inputs, targets in retain_loader:
            inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)

            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

        if (ep == epochs-2):
            vision_confuser(net , 0.005) # increase model robustness before last training epoch

        scheduler.step()

    net.eval()
    print("Seif finished")
    return net

# Sebastian - 4th place

In [ ]:
def kl_loss_fn(outputs, dist_target):
    kl_loss = F.kl_div(torch.log_softmax(outputs, dim=1), dist_target, log_target=True, reduction='batchmean')
    return kl_loss

def entropy_loss_fn(outputs, labels, dist_target, class_weights):
    ce_loss = F.cross_entropy(outputs, labels, weight=class_weights)
    entropy_dist_target = torch.sum(-torch.exp(dist_target) * dist_target, dim=1)
    entropy_outputs = torch.sum(-torch.softmax(outputs, dim=1) * torch.log_softmax(outputs, dim=1), dim=1)
    entropy_loss = F.mse_loss(entropy_outputs, entropy_dist_target)
    return ce_loss + entropy_loss

In [ ]:
def sebastian(
    net,
    retain_loader,
    forget_loader,
    val_loader,
    class_weights=None,
):
    """Simple unlearning by finetuning."""
    epochs = 3.2
    max_iters = int(len(retain_loader) * epochs)
    optimizer = optim.SGD(net.parameters(), lr=0.0005,
                      momentum=0.9, weight_decay=5e-4)
    initial_net = deepcopy(net)

    net.train()
    initial_net.eval()

    def prune_model(net, amount=0.95, rand_init=True):
        # Modules to prune
        modules = list()
        for k, m in enumerate(net.modules()):
            if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
                modules.append((m, 'weight'))
                if m.bias is not None:
                    modules.append((m, 'bias'))

        # Prune criteria
        prune.global_unstructured(
            modules,
            #pruning_method=prune.RandomUnstructured,
            pruning_method=prune.L1Unstructured,
            amount=amount,
        )

        # Perform the prune
        for k, m in enumerate(net.modules()):
            if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
                prune.remove(m, 'weight')
                if m.bias is not None:
                    prune.remove(m, 'bias')

        # Random initialization
        if rand_init:
            for k, m in enumerate(net.modules()):
                if isinstance(m, nn.Conv2d):
                    mask = m.weight == 0
                    c_in = mask.shape[1]
                    k = 1/(c_in*mask.shape[2]*mask.shape[3])
                    randinit = (torch.rand_like(m.weight)-0.5)*2*sqrt(k)
                    m.weight.data[mask] = randinit[mask]
                if isinstance(m, nn.Linear):
                    mask = m.weight == 0
                    c_in = mask.shape[1]
                    k = 1/c_in
                    randinit = (torch.rand_like(m.weight)-0.5)*2*sqrt(k)
                    m.weight.data[mask] = randinit[mask]

    num_iters = 0
    running = True
    prune_amount = 0.99
    prune_model(net, prune_amount, True)
    while running:
        net.train()
        for inputs, targets in retain_loader:
            inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)

            # Get target distribution
            with torch.no_grad():
                original_outputs = initial_net(inputs)
                preds = torch.log_softmax(original_outputs, dim=1)

            optimizer.zero_grad()
            outputs = net(inputs)
            loss = entropy_loss_fn(outputs, targets, preds, class_weights)
            loss.backward()
            optimizer.step()

            num_iters += 1
            # Stop at max iters
            if num_iters > max_iters:
                running = False
                break

    net.eval()
    print("Sebastian finished")
    return net

# Amnesiacs - 6th place

# SHAP values and Accuracy function

In [ ]:
def calculate_shap(model, test_set):
    images = [test_set[i][0] for i in range(len(test_set))]
    class_0_images = [test_set[i][0] for i in range(len(test_set)) if test_set[i][1] == 0]
    class_1_images = [test_set[i][0] for i in range(len(test_set)) if test_set[i][1] == 1]
    class_2_images = [test_set[i][0] for i in range(len(test_set)) if test_set[i][1] == 2]

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()  # Tryb ewaluacji

    # Konwersja listy obrazów na tensor i przeniesienie na CPU
    def preprocess(images):
        images = torch.stack(images).cpu().numpy()  # (batch, C, H, W)
        images = np.transpose(images, (0, 2, 3, 1))  # (batch, H, W, C)
        images = (images * 255).astype(np.uint8)  # Konwersja do uint8
        return images

    test_images = preprocess(images)
    class_0_images = preprocess(class_0_images)
    class_1_images = preprocess(class_1_images)
    class_2_images = preprocess(class_2_images)

    def model_predict(input_data):
        """Przetwarzanie wsadami (batch-wise) dla SHAP"""
        input_tensor = torch.tensor(input_data, dtype=torch.float32).to(device) / 255.0
        input_tensor = input_tensor.permute(0, 3, 1, 2)  # (batch, C, H, W)

        batch_size = 128
        outputs = []

        for i in range(0, len(input_tensor), batch_size):
            batch = input_tensor[i:i + batch_size]
            with torch.no_grad():
                output = model(batch)
                probs = torch.nn.functional.softmax(output, dim=1)
            outputs.append(probs.cpu().numpy())

        return np.concatenate(outputs, axis=0)

    # Tworzymy maskera SHAP
    masker = shap.maskers.Image("inpaint_telea", test_images[0].shape)

    # Tworzymy Explainera
    explainer = shap.Explainer(model_predict, masker)

    # Obliczamy wartości SHAP dla każdej klasy osobno
    shap_values0 = explainer(class_0_images[:100])
    shap_mean0 = shap_values0.values.sum(axis=(0, 1, 2))

    shap_values1 = explainer(class_1_images[:100])
    shap_mean1 = shap_values1.values.sum(axis=(0, 1, 2))

    """shap_values2 = explainer(class_2_images)
    shap_mean2 = shap_values2.values.sum(axis=(0, 1, 2))"""

    return np.array([shap_mean0, shap_mean1])

def shap_diffrence(previous_shap, next_shap):
  diffrence_shap = np.mean(np.array(next_shap) - np.array(previous_shap))

  return diffrence_shap

In [ ]:
initial_shap_values = calculate_shap(model, test_set)
print(initial_shap_values.shape)

In [ ]:
print(initial_shap_values[0][0][0])
print(initial_shap_values[0][1][0])
print(initial_shap_values[1][0][0])
print(initial_shap_values[1][1][0])
#print(initial_shap_values)

In [ ]:
def accuracy(net, loader):
    """Return accuracy on a dataset given by the data loader."""
    correct = 0
    total = 0
    for inputs, targets in loader:
        inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
        outputs = net(inputs)
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
    return correct / total


print(f"Train set accuracy: {100.0 * accuracy(model, train_loader):0.1f}%")
print(f"Test set accuracy: {100.0 * accuracy(model, test_loader):0.1f}%")

# Unlearning: Scenario A

In [ ]:
import copy

rt_model = copy.deepcopy(model)
scenario_0_rt = []
scenario_0_fau = []
scenario_0_koo = []
scenario_0_sei = []
scenario_0_seb = []
scenario_0_amn = []
for i in range(5):
  tmp_class_indices = class_indices.copy()
  fau_model = copy.deepcopy(model)
  koo_model = copy.deepcopy(model)
  sei_model = copy.deepcopy(model)
  seb_model = copy.deepcopy(model)
  amn_model = copy.deepcopy(model)
  forget_loader, retain_loader, tmp_class_indices = data_to_forget(train_set, tmp_class_indices, 0, 5000)
  print("Fauchan")
  fau_model = fauchan(fau_model, retain_loader, forget_loader, val_loader)
  print("Kookmin")
  koo_model = kookmin(koo_model, retain_loader, forget_loader, val_loader)
  print("Seif")
  sei_model = seif(sei_model, retain_loader, forget_loader, val_loader)
  print("Sebastian")
  seb_model = sebastian(seb_model, retain_loader, forget_loader, val_loader)
  print("Amnesiacs")
  amn_model = amnesiacs(amn_model, retain_loader, forget_loader, val_loader)
  print("Full retraining")
  rt_model = full_retraining(retain_loader)
  koo_shap = calculate_shap(koo_model, test_set)
  sei_shap = calculate_shap(sei_model, test_set)
  seb_shap = calculate_shap(seb_model, test_set)
  amn_shap = calculate_shap(amn_model, test_set)
  rt_shap = calculate_shap(rt_model, test_set)
  fau_shap = calculate_shap(fau_model, test_set)
  accuracy_rt_all = accuracy(rt_model, test_loader)
  accuracy_fau_all = accuracy(fau_model, test_loader)
  accuracy_koo_all = accuracy(koo_model, test_loader)
  accuracy_sei_all = accuracy(sei_model, test_loader)
  accuracy_seb_all = accuracy(seb_model, test_loader)
  accuracy_amn_all = accuracy(amn_model, test_loader)
  accuracy_rt_0 = accuracy(rt_model, test_loader_0)
  accuracy_fau_0 = accuracy(fau_model, test_loader_0)
  accuracy_koo_0 = accuracy(koo_model, test_loader_0)
  accuracy_sei_0 = accuracy(sei_model, test_loader_0)
  accuracy_seb_0 = accuracy(seb_model, test_loader_0)
  accuracy_amn_0 = accuracy(amn_model, test_loader_0)
  accuracy_rt_no_0 = accuracy(rt_model, test_loader_no_0)
  accuracy_fau_no_0 = accuracy(fau_model, test_loader_no_0)
  accuracy_koo_no_0 = accuracy(koo_model, test_loader_no_0)
  accuracy_sei_no_0 = accuracy(sei_model, test_loader_no_0)
  accuracy_seb_no_0 = accuracy(seb_model, test_loader_no_0)
  accuracy_amn_no_0 = accuracy(amn_model, test_loader_no_0)
  scenario_0_rt.append((accuracy_rt_all, accuracy_rt_0, accuracy_rt_no_0))
  scenario_0_fau.append((accuracy_fau_all, accuracy_fau_0, accuracy_fau_no_0))
  scenario_0_koo.append((accuracy_koo_all, accuracy_koo_0, accuracy_koo_no_0))
  scenario_0_sei.append((accuracy_sei_all, accuracy_sei_0, accuracy_sei_no_0))
  scenario_0_seb.append((accuracy_seb_all, accuracy_seb_0, accuracy_seb_no_0))
  scenario_0_amn.append((accuracy_amn_all, accuracy_amn_0, accuracy_amn_no_0))
print(scenario_0_rt)
print(scenario_0_fau)
print(scenario_0_koo)
print(scenario_0_sei)
print(scenario_0_seb)
print(scenario_0_amn)


In [ ]:
import copy

rt_model = copy.deepcopy(model)
fau_model = copy.deepcopy(model)
koo_model = copy.deepcopy(model)
sei_model = copy.deepcopy(model)
seb_model = copy.deepcopy(model)
amn_model = copy.deepcopy(model)
tmp_class_indices = class_indices.copy()
scenario_0_rt = []
scenario_0_fau = []
scenario_0_koo = []
scenario_0_sei = []
scenario_0_seb = []
scenario_0_amn = []

for i in range(50):
  forget_loader, retain_loader, tmp_class_indices = data_to_forget(train_set, tmp_class_indices, 0, 100)
  print("Fauchan")
  fau_model = fauchan(fau_model, retain_loader, forget_loader, val_loader)
  print("Kookmin")
  koo_model = kookmin(koo_model, retain_loader, forget_loader, val_loader)
  print("Seif")
  sei_model = seif(sei_model, retain_loader, forget_loader, val_loader)
  print("Sebastian")
  seb_model = sebastian(seb_model, retain_loader, forget_loader, val_loader)
  print("Amnesiacs")
  amn_model = amnesiacs(amn_model, retain_loader, forget_loader, val_loader)
  print("Full retraining")
  rt_model = full_retraining(retain_loader)
  koo_shap = calculate_shap(koo_model, test_set)
  sei_shap = calculate_shap(sei_model, test_set)
  seb_shap = calculate_shap(seb_model, test_set)
  amn_shap = calculate_shap(amn_model, test_set)
  rt_shap = calculate_shap(rt_model, test_set)
  fau_shap = calculate_shap(fau_model, test_set)
  accuracy_rt_all = accuracy(rt_model, test_loader)
  accuracy_fau_all = accuracy(fau_model, test_loader)
  accuracy_koo_all = accuracy(koo_model, test_loader)
  accuracy_sei_all = accuracy(sei_model, test_loader)
  accuracy_seb_all = accuracy(seb_model, test_loader)
  accuracy_amn_all = accuracy(amn_model, test_loader)
  accuracy_rt_0 = accuracy(rt_model, test_loader_0)
  accuracy_fau_0 = accuracy(fau_model, test_loader_0)
  accuracy_koo_0 = accuracy(koo_model, test_loader_0)
  accuracy_sei_0 = accuracy(sei_model, test_loader_0)
  accuracy_seb_0 = accuracy(seb_model, test_loader_0)
  accuracy_amn_0 = accuracy(amn_model, test_loader_0)
  accuracy_rt_no_0 = accuracy(rt_model, test_loader_no_0)
  accuracy_fau_no_0 = accuracy(fau_model, test_loader_no_0)
  accuracy_koo_no_0 = accuracy(koo_model, test_loader_no_0)
  accuracy_sei_no_0 = accuracy(sei_model, test_loader_no_0)
  accuracy_seb_no_0 = accuracy(seb_model, test_loader_no_0)
  accuracy_amn_no_0 = accuracy(amn_model, test_loader_no_0)
  scenario_0_rt.append((accuracy_rt_all, accuracy_rt_0, accuracy_rt_no_0, rt_shap))
  scenario_0_fau.append((accuracy_fau_all, accuracy_fau_0, accuracy_fau_no_0, fau_shap))
  scenario_0_koo.append((accuracy_koo_all, accuracy_koo_0, accuracy_koo_no_0, koo_shap))
  scenario_0_sei.append((accuracy_sei_all, accuracy_sei_0, accuracy_sei_no_0, sei_shap))
  scenario_0_seb.append((accuracy_seb_all, accuracy_seb_0, accuracy_seb_no_0, seb_shap))
  scenario_0_amn.append((accuracy_amn_all, accuracy_amn_0, accuracy_amn_no_0, amn_shap))
  print("Finish"+str(i))



# Unlearning: Scenario B

In [ ]:
import copy

rt_model = copy.deepcopy(model)
fau_model = copy.deepcopy(model)
koo_model = copy.deepcopy(model)
sei_model = copy.deepcopy(model)
seb_model = copy.deepcopy(model)
amn_model = copy.deepcopy(model)
tmp_class_indices = class_indices.copy()
scenario_0_rt = []
scenario_0_fau = []
scenario_0_koo = []
scenario_0_sei = []
scenario_0_seb = []
scenario_0_amn = []

for i in range(50):
  forget_loader, retain_loader, tmp_class_indices = data_to_forget(train_set, tmp_class_indices, -1, 100)
  print("Fauchan")
  fau_model = fauchan(fau_model, retain_loader, forget_loader, val_loader)
  print("Kookmin")
  koo_model = kookmin(koo_model, retain_loader, forget_loader, val_loader)
  print("Seif")
  sei_model = seif(sei_model, retain_loader, forget_loader, val_loader)
  print("Sebastian")
  seb_model = sebastian(seb_model, retain_loader, forget_loader, val_loader)
  print("Amnesiacs")
  amn_model = amnesiacs(amn_model, retain_loader, forget_loader, val_loader)
  print("Full retraining")
  rt_model = full_retraining(retain_loader)
  koo_shap = calculate_shap(koo_model, test_set)
  sei_shap = calculate_shap(sei_model, test_set)
  seb_shap = calculate_shap(seb_model, test_set)
  amn_shap = calculate_shap(amn_model, test_set)
  rt_shap = calculate_shap(rt_model, test_set)
  fau_shap = calculate_shap(fau_model, test_set)
  accuracy_rt_all = accuracy(rt_model, test_loader)
  accuracy_fau_all = accuracy(fau_model, test_loader)
  accuracy_koo_all = accuracy(koo_model, test_loader)
  accuracy_sei_all = accuracy(sei_model, test_loader)
  accuracy_seb_all = accuracy(seb_model, test_loader)
  accuracy_amn_all = accuracy(amn_model, test_loader)
  accuracy_rt_0 = accuracy(rt_model, test_loader_0)
  accuracy_fau_0 = accuracy(fau_model, test_loader_0)
  accuracy_koo_0 = accuracy(koo_model, test_loader_0)
  accuracy_sei_0 = accuracy(sei_model, test_loader_0)
  accuracy_seb_0 = accuracy(seb_model, test_loader_0)
  accuracy_amn_0 = accuracy(amn_model, test_loader_0)
  accuracy_rt_no_0 = accuracy(rt_model, test_loader_no_0)
  accuracy_fau_no_0 = accuracy(fau_model, test_loader_no_0)
  accuracy_koo_no_0 = accuracy(koo_model, test_loader_no_0)
  accuracy_sei_no_0 = accuracy(sei_model, test_loader_no_0)
  accuracy_seb_no_0 = accuracy(seb_model, test_loader_no_0)
  accuracy_amn_no_0 = accuracy(amn_model, test_loader_no_0)
  scenario_0_rt.append((accuracy_rt_all, accuracy_rt_0, accuracy_rt_no_0, rt_shap))
  scenario_0_fau.append((accuracy_fau_all, accuracy_fau_0, accuracy_fau_no_0, fau_shap))
  scenario_0_koo.append((accuracy_koo_all, accuracy_koo_0, accuracy_koo_no_0, koo_shap))
  scenario_0_sei.append((accuracy_sei_all, accuracy_sei_0, accuracy_sei_no_0, sei_shap))
  scenario_0_seb.append((accuracy_seb_all, accuracy_seb_0, accuracy_seb_no_0, seb_shap))
  scenario_0_amn.append((accuracy_amn_all, accuracy_amn_0, accuracy_amn_no_0, amn_shap))
  print("Finish"+str(i))



# Save results

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pickle

save_path = "/content/drive/My Drive/PPRAI/rt0.pkl"
with open(save_path, "wb") as file:
    pickle.dump(scenario_0_rt, file)
print("Plik zapisany w:", save_path)

In [ ]:
save_path = "/content/drive/My Drive/PPRAI/fau0.pkl"
with open(save_path, "wb") as file:
    pickle.dump(scenario_0_fau, file)
print("Plik zapisany w:", save_path)

In [ ]:
save_path = "/content/drive/My Drive/PPRAI/koo0.pkl"
with open(save_path, "wb") as file:
    pickle.dump(scenario_0_koo, file)
print("Plik zapisany w:", save_path)

In [ ]:
save_path = "/content/drive/My Drive/PPRAI/sei0.pkl"
with open(save_path, "wb") as file:
    pickle.dump(scenario_0_sei, file)
print("Plik zapisany w:", save_path)

In [ ]:
save_path = "/content/drive/My Drive/PPRAI/seb0.pkl"
with open(save_path, "wb") as file:
    pickle.dump(scenario_0_seb, file)
print("Plik zapisany w:", save_path)

In [ ]:
save_path = "/content/drive/My Drive/PPRAI/amn0.pkl"
with open(save_path, "wb") as file:
    pickle.dump(scenario_0_amn, file)
print("Plik zapisany w:", save_path)

In [ ]:
with open(save_path, "rb") as file:
    loaded_data = pickle.load(file)

print(loaded_data[29])